In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os

In [28]:
pd.options.display.max_rows = 160
pd.options.display.max_columns = 85

#### read in geocoded

In [3]:
df = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/new_totalprojects.csv')
df.head()

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
0,1,World,M,100.0,A,"1707 Edgerly Ave, Albany, Georgia, 31707","1707 Edgerly Ave, Albany, GA, 31707, USA",1707 Edgerly Ave,PointAddress,NaN,NaN,"1707 Edgerly Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1707.0,NaN,NaN,NaN,L,NaN,NaN,Edgerly,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1707 Edgerly Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,4208.0,USA,ENG,0.0,-84.191282,31.573277,-84.191292,31.573530,-84.192292,-84.190292,31.572530,31.574530,NaN,1707 Edgerly Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",-84.191293,31.573502,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,2,World,M,98.7,A,"703 Dartmouth Ln, Albany, Georgia, 31705","703 Dartmouth Ln, Albany, GA, 31705, USA",703 Dartmouth Ln,PointAddress,NaN,NaN,"703 Dartmouth Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,703.0,NaN,NaN,NaN,R,NaN,NaN,Dartmouth,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703 Dartmouth Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1211.0,USA,ENG,0.0,-84.136801,31.588103,-84.136802,31.588326,-84.137802,-84.135801,31.587326,31.589326,NaN,703 Dartmouth,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",-84.136794,31.588278,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,3,World,M,100.0,A,"1115 W Waddell Ave, Albany, Georgia, 31707","1115 W Waddell Ave, Albany, GA, 31707, USA",1115 W Waddell Ave,PointAddress,NaN,NaN,"1115 W Waddell Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1115.0,NaN,NaN,NaN,L,W,NaN,Waddell,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1115 W Waddell Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5115.0,USA,ENG,0.0,-84.178174,31.566717,-84.178184,31.567059,-84.179183,-84.177183,31.566059,31.568059,NaN,1115 W. Waddell Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",-84.178208,31.566964,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,4,World,M,98.7,A,"313 Forrestal Ln, Albany, Georgia, 31705","313 Forrestal Ln, Albany, GA, 31705, USA",313 Forrestal Ln,PointAddress,NaN,NaN,"313 Forrestal Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,313.0,NaN,NaN,NaN,L,NaN,NaN,Forrestal,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313 Forrestal Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,2508.0,USA,ENG,0.0,-84.082134,31.578470,-84.082442,31.578579,-84.083442,-84.081441,31.577579,31.579579,NaN,313 Forrestal,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",-84.082410,31.578511,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,5,World,M,100.0,A,"1125 Joel Dr, Albany, Georgia, 31707","1125 Joel Dr, Albany, GA, 31707, USA",1125 Joel Dr,PointAddress,NaN,NaN,"1125 Joel Dr, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1125.0,NaN,NaN,NaN,L,NaN,NaN,Joel,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1125 Joel Dr,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5021.0,USA,ENG,0.0,-84.207624,31.561222,-84.207901,31.561119,-84.208901,-84.206901,31.560119,31.562119,NaN,11

#### read in originals

In [4]:
xlsx = pd.ExcelFile('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)
df_original.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 Joel Drive,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


#### fix address script

In [5]:
df1_fixed = pd.read_csv('/Users/william/Documents/Summer2019/AlbanyHousing/code/albany-housing-analytics/preprocessing/HOME_2007-2017_fixed_address.csv')
df2_fixed = pd.read_csv('/Users/william/Documents/Summer2019/AlbanyHousing/code/albany-housing-analytics/preprocessing/CDBG_2007-2017_fixed_address.csv')

In [6]:
df1_fixed = df1_fixed.drop('Unnamed: 0', axis = 1)
df2_fixed = df2_fixed.drop('Unnamed: 0', axis = 1)

In [7]:
df1_fixed.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [8]:
df2_fixed.columns

Index(['Parcel ID', 'Address', 'City', 'State', 'XY Coordinates', 'Plan Year',
       'IDIS Project', 'Project', 'IDIS Activity ID', 'Activity Status',
       'Program', 'Funded Amount', 'Draw Amount', 'Balance'],
      dtype='object')

#### comparisons

In [10]:
compare_address = (df1_fixed['Address'] != df1_original['Address'].str.upper())
df1_original[compare_address].head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 Joel Drive,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [13]:
df1_fixed[compare_address].head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [14]:
df.shape

(850, 85)

In [15]:
df1 = df[df['Program'] == "HOME"]
df2 = df[df['Program'] == "CDBG"]

In [29]:
df1_fixed[(df1['ShortLabel'].str.upper() != df1_fixed['Address'])]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
24,0000W/00003/006,1403 GREENBRIAR CT,Albany,GA,"31.6040376, -84.1206097",2008,9,Homeowner Rehabilitation,671,Completed,HOME,32802.78,32802.78,0.00
28,0000K/00017/008,1602 ESTELLE AVE,Albany,GA,"31.5976214, -84.1119695",2009,5,Homeowner Rehabilitation,783,Completed,HOME,40580.36,40580.36,0.00
58,0000K/00045/015,1410 S CAROLINA AVE,Albany,GA,"31.600016, -84.116951",2010,24,CHDO,934,Completed,HOME,33081.99,33081.99,0.00
59,0000K/00045/005,1411 S CAROLINA AVE,Albany,GA,"31.6003837, -84.1168707",2010,24,CHDO,935,Completed,HOME,33081.99,33081.99,0.00
60,0000K/00045/014,1412 S CAROLINA AVE,Albany,GA,"31.600003, -84.116732",2010,24,CHDO,936,Completed,HOME,33081.99,33081.99,0.00
61,0000K/00045/004,1413 S CAROLINA AVE,Albany,GA,"31.6003678, -84.116681",2010,24,CHDO,937,Completed,HOME,33081.99,33081.99,0.00
62,0000K/00045/013,1414 S CAROLINA AVE,Albany,GA,"31.599993, -84.116512",2010,24,CHDO,1001,Completed,HOME,33081.99,33081.99,0.00
63,0000K/00045/003,1415 S CAROLINA AVE,Albany,GA,"31.6003581, -84.1164775",2010,24,CHDO,933,Completed,HOME,33081.99,33081.99,0.00
64,0000K/00045/003,1415 S CAROLINA AVE,Albany,GA,"31.6003581, -84.1164775",2010,24,CHDO,1002,Completed,HOME,33081.99,33081.99,0.00
65,0000K/00045/012,1416 S CAROLINA AVE,Albany,GA,"31.599988, -84.116308",2010,24,CHDO,1003,Completed,HOME,33081.95,33081.95,0.00


In [30]:
df1[(df1['ShortLabel'].str.upper() != df1_fixed['Address'])]

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
24,25,World,M,97.93,A,"1403 Greenbrier Ct, Albany, Georgia, 31705","1403 Greenbrier Ct, Albany, GA, 31705, USA",1403 Greenbrier Ct,PointAddress,NaN,NaN,"1403 Greenbrier Ct, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,1403.0,NaN,NaN,NaN,L,NaN,NaN,Greenbrier,Ct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1403 Greenbrier Ct,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1098.0,USA,ENG,0.0,-84.120335,31.603979,-84.120610,31.604085,-84.121611,-84.119610,31.603085,31.605085,NaN,1403 Greenbriar Court,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000W/00003/006,1403 Greenbriar Court,Albany,GA,"31.6040376, -84.1206097",-84.120610,31.604038,2008,9,Homeowner Rehabilitation,671,Completed,HOME,32802.78,32802.78,0.0
28,29,World,M,97.83,A,"1602 Estelle St, Albany, Georgia, 31705","1602 Estelle St, Albany, GA, 31705, USA",1602 Estelle St,PointAddress,NaN,NaN,"1602 Estelle St, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,1602.0,NaN,NaN,NaN,R,NaN,NaN,Estelle,St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1602 Estelle St,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1422.0,USA,ENG,0.0,-84.112331,31.597640,-84.111943,31.597659,-84.112944,-84.110943,31.596659,31.598659,NaN,1602 Estelle Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000K/00017/008,1602 Estelle Avenue,Albany,GA,"31.5976214, -84.1119695",-84.111970,31.597621,2009,5,Homeowner Rehabilitation,783,Completed,HOME,40580.36,40580.36,0.0
58,59,World,M,100.00,A,"1410 South Carolina Ave, Albany, Georgia, 31705","1410 South Carolina Ave, Albany, GA, 31705, USA",1410 South Carolina Ave,PointAddress,NaN,NaN,"1410 South Carolina Ave, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,1410.0,NaN,NaN,NaN,L,NaN,NaN,South Carolina,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1410 South Carolina Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1064.0,USA,ENG,0.0,-84.116906,31.600200,-84.116952,31.600017,-84.117952,-84.115952,31.599017,31.601017,NaN,1410 South Carolina Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000K/00045/015,1410 South Carolina Avenue,Albany,GA,"31.600016, -84.116951",-84.116951,31.600016,2010,24,CHDO,934,Completed,HOME,33081.99,33081.99,0.0
59,60,World,M,100.00,A,"1411 South Carolina Ave, Albany, Georgia, 31705","1411 South Carolina Ave, Albany, GA, 31705, USA",1411 South Carolina Ave,PointAddress,NaN,NaN,"1411 South Carolina Ave, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,1411.0,NaN,NaN,NaN,R,NaN,NaN,South Carolina,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1411 South Carolina Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1005.0,USA,ENG,0.0,-84.116917,31.600255,-84.116871,31.600458,-84.117871,-84.115871,31.599458,31.601458,NaN,1411 South Carolina Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000K/00045/005,1411 South Carolina Avenue,Albany,GA,"31.6003837, -84.1168707",-84.116871,31.600384,2010,24,CHDO,935,Completed,HOME,33081.99,33081.99,0.0
60,61,World,M,100.00,A,"1412 South Carolina Ave, Albany, Georgia, 31705","1412 South Carolina Ave, Albany, GA, 31705, USA",1412 South Carolina Ave,PointAddress,NaN,NaN,"1412 South Carolina Ave, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,1412.0,NaN,NaN,NaN,L,NaN,NaN,South Carolina,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1412 South Carolina Ave,NaN,Na

In [31]:
df1.head()

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
0,1,World,M,100.0,A,"1707 Edgerly Ave, Albany, Georgia, 31707","1707 Edgerly Ave, Albany, GA, 31707, USA",1707 Edgerly Ave,PointAddress,NaN,NaN,"1707 Edgerly Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1707.0,NaN,NaN,NaN,L,NaN,NaN,Edgerly,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1707 Edgerly Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,4208.0,USA,ENG,0.0,-84.191282,31.573277,-84.191292,31.573530,-84.192292,-84.190292,31.572530,31.574530,NaN,1707 Edgerly Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",-84.191293,31.573502,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,2,World,M,98.7,A,"703 Dartmouth Ln, Albany, Georgia, 31705","703 Dartmouth Ln, Albany, GA, 31705, USA",703 Dartmouth Ln,PointAddress,NaN,NaN,"703 Dartmouth Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,703.0,NaN,NaN,NaN,R,NaN,NaN,Dartmouth,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703 Dartmouth Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1211.0,USA,ENG,0.0,-84.136801,31.588103,-84.136802,31.588326,-84.137802,-84.135801,31.587326,31.589326,NaN,703 Dartmouth,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",-84.136794,31.588278,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,3,World,M,100.0,A,"1115 W Waddell Ave, Albany, Georgia, 31707","1115 W Waddell Ave, Albany, GA, 31707, USA",1115 W Waddell Ave,PointAddress,NaN,NaN,"1115 W Waddell Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1115.0,NaN,NaN,NaN,L,W,NaN,Waddell,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1115 W Waddell Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5115.0,USA,ENG,0.0,-84.178174,31.566717,-84.178184,31.567059,-84.179183,-84.177183,31.566059,31.568059,NaN,1115 W. Waddell Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",-84.178208,31.566964,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,4,World,M,98.7,A,"313 Forrestal Ln, Albany, Georgia, 31705","313 Forrestal Ln, Albany, GA, 31705, USA",313 Forrestal Ln,PointAddress,NaN,NaN,"313 Forrestal Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,313.0,NaN,NaN,NaN,L,NaN,NaN,Forrestal,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313 Forrestal Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,2508.0,USA,ENG,0.0,-84.082134,31.578470,-84.082442,31.578579,-84.083442,-84.081441,31.577579,31.579579,NaN,313 Forrestal,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",-84.082410,31.578511,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,5,World,M,100.0,A,"1125 Joel Dr, Albany, Georgia, 31707","1125 Joel Dr, Albany, GA, 31707, USA",1125 Joel Dr,PointAddress,NaN,NaN,"1125 Joel Dr, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1125.0,NaN,NaN,NaN,L,NaN,NaN,Joel,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1125 Joel Dr,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5021.0,USA,ENG,0.0,-84.207624,31.561222,-84.207901,31.561119,-84.208901,-84.206901,31.560119,31.562119,NaN,11

In [38]:
df['Addr_type'].value_counts()

PointAddress        653
StreetAddress       102
Subaddress           69
Locality             13
StreetAddressExt     11
StreetName            1
StreetInt             1
Name: Addr_type, dtype: int64

In [40]:
df_utilities = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')

In [43]:
df_utilities.head()

,Premise,Premise Address,City,State,Customer,Service Type,Service Number,Service Rate,Consumption,Days of Service,Unit of Measure,Charge Amount,Charge Date,Month,Year
0,10002.0,212 ACORN ST,ALBANY,GA,10002.0,DELC,100.0,216,340.0,30.0,KWH,79.85,2012-01-09,Jan,2012
1,10002.0,212 ACORN ST,ALBANY,GA,10002.0,NaN,150.0,NaN,NaN,NaN,NaN,17.12,2012-01-09,Jan,2012
2,10002.0,212 ACORN ST,ALBANY,GA,10002.0,CGAS,200.0,220,37.0,30.0,CCF,47.13,2012-01-09,Jan,2012
3,10002.0,212 ACORN ST,ALBANY,GA,10002.0,CWTR,300.0,230,1.0,30.0,TGAL,9.67,2012-01-09,Jan,2012
4,10002.0,212 ACORN ST,ALBANY,GA,10002.0,CSWR,400.0,250,1.0,30.0,NONE,10.93,2012-01-09,Jan,2012


In [48]:
df1_fixed[(df1_fixed['Address'].isin(df_utilities['Premise Address']))]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance


In [50]:
df1_fixed['Address'].isin(df_utilities['Premise Address'])

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

In [52]:
df1_fixed.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [63]:
df1_fixed.iloc[-1,0] = 1
df1_fixed.tail()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
281,00306/00001/08D,2010 W BROAD AVE APT 24,Albany,GA,"31.5774669, -84.2011111",2017,11,Tenant Based Rental Assistance,1495,Open,HOME,4757.00,2422.38,2334.62
282,00306/00001/08D,2010 W BROAD AVE APT 71,Albany,GA,"31.5774669, -84.2011111",2017,11,Tenant Based Rental Assistance,1540,Open,HOME,3547.20,1322.94,2224.26
283,00306/00001/08D,2010 W BROAD AVE APT 79,Albany,GA,"31.5774669, -84.2011111",2017,11,Tenant Based Rental Assistance,1492,Open,HOME,4390.53,4390.53,0.00
284,0000N/00027/08F,1404 WHISPERING PINES RD APT G5,Albany,GA,"31.6032603, -84.184194",2017,11,Tenant Based Rental Assistance,1489,Open,HOME,5639.00,4418.53,1220.47
285,1,1014 RANDOLPH ST,Albany,GA,"31.5440891, -84.1757622",2017,11,Tenant Based Rental Assistance,1491,Completed,HOME,4474.53,4474.53,0.00


In [56]:
df_utilities[df_utilities['Premise Address'].str.contains("Edgerly Ave")]

ValueError: cannot index with vector containing NA / NaN values